In [27]:
# Exercise 44 - Multiple inpur RDDs (complexity similar to exam)

In [23]:
sc
inMovies = "/data/students/bigdata-01QYD/ex_data/Ex44/data/movies.txt"
inWatchedMovies = "/data/students/bigdata-01QYD/ex_data/Ex44/data/watchedmovies.txt"
inPreferences = "/data/students/bigdata-01QYD/ex_data/Ex44/data/preferences.txt"

threshold = 0.2

In [3]:
moviesRDD = sc.textFile(inMovies)
watchedMoviesRDD = sc.textFile(inWatchedMovies)
preferencesRDD = sc.textFile(inPreferences)

In [4]:
moviesRDD.top(2)

['movie9,Sudden Death (1995),Action', 'movie8,Tom and Huck (1995),Adventure']

In [5]:
watchedMoviesRDD.top(2)

['user2,movie6,201610081800,201610081845',
 'user2,movie4,201611051100,201611051105']

In [6]:
preferencesRDD.top(2)

['user2,Action', 'user1,Comedy']

In [7]:
movieGenreMap = moviesRDD.map(lambda record: (record.split(',')[0],record.split(',')[2]))
movieGenreMap.collect()

[('movie1', 'Animation'),
 ('movie2', 'Adventure'),
 ('movie3', 'Comedy'),
 ('movie4', 'Comedy'),
 ('movie5', 'Comedy'),
 ('movie6', 'Action'),
 ('movie7', 'Comedy'),
 ('movie8', 'Adventure'),
 ('movie9', 'Action'),
 ('movie10', 'Action')]

In [8]:
userWatchedMap = watchedMoviesRDD.map(lambda record: (record.split(',')[1],record.split(',')[0]))
userWatchedMap.collect()

[('movie1', 'user1'),
 ('movie3', 'user1'),
 ('movie4', 'user1'),
 ('movie5', 'user1'),
 ('movie6', 'user2'),
 ('movie3', 'user2'),
 ('movie4', 'user2')]

In [9]:
userMovieGenre = userWatchedMap.join(movieGenreMap)
userMovieGenre.collect()

[('movie5', ('user1', 'Comedy')),
 ('movie1', ('user1', 'Animation')),
 ('movie4', ('user1', 'Comedy')),
 ('movie4', ('user2', 'Comedy')),
 ('movie3', ('user1', 'Comedy')),
 ('movie3', ('user2', 'Comedy')),
 ('movie6', ('user2', 'Action'))]

In [10]:
userGenreRDD = userMovieGenre.values()

In [11]:
userGenreRDD.collect()

[('user1', 'Comedy'),
 ('user1', 'Animation'),
 ('user1', 'Comedy'),
 ('user2', 'Comedy'),
 ('user1', 'Comedy'),
 ('user2', 'Comedy'),
 ('user2', 'Action')]

In [16]:
userPref = preferencesRDD.map(lambda record: (record.split(',')[0],record.split(',')[1]))
userPref.collect()

[('user1', 'Animation'), ('user1', 'Comedy'), ('user2', 'Action')]

In [17]:
userGenrePref = userGenreRDD.cogroup(userPref)
userGenrePref.collect()

[('user2',
  (<pyspark.resultiterable.ResultIterable at 0x7f2a942d8910>,
   <pyspark.resultiterable.ResultIterable at 0x7f2a942d8650>)),
 ('user1',
  (<pyspark.resultiterable.ResultIterable at 0x7f2a942d8410>,
   <pyspark.resultiterable.ResultIterable at 0x7f2a942d8090>))]

In [19]:
# just for debuggin, the RDD is not changed
userGenrePref.mapValues(lambda v: (list(v[0]), list(v[1]))).collect()

[('user2', (['Comedy', 'Comedy', 'Action'], ['Action'])),
 ('user1',
  (['Comedy', 'Animation', 'Comedy', 'Comedy'], ['Animation', 'Comedy']))]

In [24]:
# Compute misleading users 
def misleading(userGenrePref):
    # Store in a local list the "small" set of liked generes
    # associated with the current user
    likedGenres = list(userGenrePref[1][1])
    
    # Iterate over the watched movies
    numWatchedMovies = 0
    notLiked = 0
    
    for watchedGenre in userGenrePref[1][0]:
        numWatchedMovies += 1
        if watchedGenre not in likedGenres:
            notLiked += 1
            
    if float(notLiked) > threshold * float(numWatchedMovies):
        return True
    else:
        return False

In [25]:
misleadingUserListRDD = userGenrePref.filter(misleading)

In [26]:
misleadingUserListRDD.mapValues(lambda v: (list(v[0]), list(v[1]))).collect()

[('user2', (['Comedy', 'Comedy', 'Action'], ['Action']))]